# Python Libraries Primer for Drug Discovery
## Prerequisites for the Alzheimer's Drug Discovery Project

This notebook introduces the key Python libraries used in computational drug discovery. Work through these examples to understand each library before diving into the main project.

**Libraries Covered:**
1. **NumPy** — Numerical computing
2. **Pandas** — Data manipulation
3. **Matplotlib & Seaborn** — Visualization
4. **RDKit** — Chemistry/molecular analysis
5. **Scikit-learn** — Machine learning

**Learning Objectives:**
- Load and manipulate data with pandas
- Perform numerical operations with numpy
- Create visualizations
- Parse molecular structures with RDKit
- Build basic ML models

---
# 0. Setup: Install Required Libraries

In [ ]:
# Install all required libraries using uv (faster!)
!pip install uv -q
!uv pip install --system numpy pandas matplotlib seaborn rdkit scikit-learn --quiet
print("✓ All libraries installed!")

---
# 1. NumPy — Numerical Computing

NumPy is the foundation for numerical computing in Python. It provides fast array operations.

In [ ]:
import numpy as np
print(f"NumPy version: {np.__version__}")

## 1.1 Creating Arrays

In [ ]:
# Create arrays from lists
ic50_values = np.array([100, 500, 1000, 5000, 10000, 50000])
print("IC50 values (nM):", ic50_values)
print("Shape:", ic50_values.shape)
print("Data type:", ic50_values.dtype)

In [ ]:
# Create special arrays
zeros = np.zeros(5)
ones = np.ones(5)
range_arr = np.arange(0, 10, 2)  # start, stop, step
linspace = np.linspace(0, 1, 5)  # start, stop, num_points

print("Zeros:", zeros)
print("Ones:", ones)
print("Range:", range_arr)
print("Linspace:", linspace)

## 1.2 Array Operations (Vectorized)

NumPy operations work on entire arrays at once — much faster than loops!

In [ ]:
# Convert IC50 (nM) to pIC50
# pIC50 = -log10(IC50 in Molar)
# IC50 in Molar = IC50 in nM * 1e-9

ic50_molar = ic50_values * 1e-9
pIC50 = -np.log10(ic50_molar)

print("IC50 (nM):", ic50_values)
print("IC50 (M):", ic50_molar)
print("pIC50:", pIC50)

In [ ]:
# Statistical operations
print(f"Mean pIC50: {np.mean(pIC50):.2f}")
print(f"Std pIC50: {np.std(pIC50):.2f}")
print(f"Min pIC50: {np.min(pIC50):.2f}")
print(f"Max pIC50: {np.max(pIC50):.2f}")

## 1.3 Boolean Indexing (Filtering)

In [ ]:
# Find "active" compounds (IC50 < 1000 nM, or pIC50 > 6)
active_mask = pIC50 > 6
print("Active mask:", active_mask)
print("Active pIC50 values:", pIC50[active_mask])
print("Number of active compounds:", np.sum(active_mask))

---
# 2. Pandas — Data Manipulation

Pandas is essential for working with tabular data (like CSV files).

In [ ]:
import pandas as pd
print(f"Pandas version: {pd.__version__}")

## 2.1 Creating DataFrames

In [ ]:
# Create a DataFrame from a dictionary
data = {
    'molecule_id': ['MOL001', 'MOL002', 'MOL003', 'MOL004', 'MOL005', 'MOL006'],
    'smiles': ['CCO', 'CCCO', 'CCCCO', 'CC(C)O', 'CCOCC', 'c1ccccc1'],
    'ic50_nM': [100, 500, 1000, 5000, 10000, 50000],
    'molecular_weight': [46.07, 60.10, 74.12, 60.10, 74.12, 78.11]
}

df = pd.DataFrame(data)
df

## 2.2 Basic DataFrame Operations

In [ ]:
# Basic info
print("Shape:", df.shape)
print("\nColumn types:")
print(df.dtypes)
print("\nBasic statistics:")
df.describe()

In [ ]:
# Access columns
print("SMILES column:")
print(df['smiles'])

In [ ]:
# Access rows by index
print("First row:")
print(df.iloc[0])
print("\nFirst 3 rows:")
df.head(3)

## 2.3 Adding New Columns

In [ ]:
# Calculate pIC50 and add as new column
df['pIC50'] = -np.log10(df['ic50_nM'] * 1e-9)

# Add bioactivity class
def classify_activity(ic50):
    if ic50 <= 1000:
        return 'active'
    elif ic50 >= 10000:
        return 'inactive'
    else:
        return 'intermediate'

df['class'] = df['ic50_nM'].apply(classify_activity)
df

## 2.4 Filtering Data

In [ ]:
# Filter for active compounds only
active_df = df[df['class'] == 'active']
print("Active compounds:")
active_df

In [ ]:
# Multiple conditions
# Active AND molecular weight < 70
filtered = df[(df['class'] == 'active') & (df['molecular_weight'] < 70)]
print("Active compounds with MW < 70:")
filtered

## 2.5 Grouping and Aggregation

In [ ]:
# Count by class
print("Compounds per class:")
print(df['class'].value_counts())

In [ ]:
# Group statistics
print("\nMean pIC50 by class:")
print(df.groupby('class')['pIC50'].mean())

## 2.6 Reading/Writing CSV Files

In [ ]:
# Save to CSV
df.to_csv('sample_bioactivity.csv', index=False)
print("Saved to sample_bioactivity.csv")

# Read back
df_loaded = pd.read_csv('sample_bioactivity.csv')
df_loaded.head()

---
# 3. Matplotlib & Seaborn — Visualization

Creating plots to understand your data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

## 3.1 Bar Plot — Class Distribution

In [ ]:
# Count plot for bioactivity classes
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='class', order=['active', 'intermediate', 'inactive'],
              palette=['green', 'orange', 'red'])
plt.title('Distribution of Bioactivity Classes')
plt.xlabel('Bioactivity Class')
plt.ylabel('Count')
plt.show()

## 3.2 Box Plot — pIC50 by Class

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='class', y='pIC50', 
            order=['active', 'intermediate', 'inactive'],
            palette=['green', 'orange', 'red'])
plt.title('pIC50 Distribution by Bioactivity Class')
plt.xlabel('Bioactivity Class')
plt.ylabel('pIC50')
plt.axhline(y=6, color='blue', linestyle='--', label='Active threshold (pIC50=6)')
plt.legend()
plt.show()

## 3.3 Scatter Plot — MW vs pIC50

In [ ]:
plt.figure(figsize=(8, 6))
colors = {'active': 'green', 'intermediate': 'orange', 'inactive': 'red'}
for cls in ['active', 'intermediate', 'inactive']:
    subset = df[df['class'] == cls]
    plt.scatter(subset['molecular_weight'], subset['pIC50'], 
                c=colors[cls], label=cls, s=100, alpha=0.7)

plt.xlabel('Molecular Weight (Da)')
plt.ylabel('pIC50')
plt.title('Chemical Space: MW vs pIC50')
plt.legend()
plt.show()

## 3.4 Histogram — Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# pIC50 distribution
axes[0].hist(df['pIC50'], bins=10, color='steelblue', edgecolor='black')
axes[0].set_xlabel('pIC50')
axes[0].set_ylabel('Frequency')
axes[0].set_title('pIC50 Distribution')

# Molecular weight distribution
axes[1].hist(df['molecular_weight'], bins=10, color='coral', edgecolor='black')
axes[1].set_xlabel('Molecular Weight (Da)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Molecular Weight Distribution')

plt.tight_layout()
plt.show()

---
# 4. RDKit — Chemistry & Molecular Analysis

RDKit is the go-to library for cheminformatics in Python.

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Draw
print("RDKit imported successfully!")

## 4.1 Parsing SMILES Strings

SMILES (Simplified Molecular Input Line Entry System) is a text representation of molecules.

In [ ]:
# Parse a SMILES string into a molecule object
smiles = "CCO"  # Ethanol
mol = Chem.MolFromSmiles(smiles)

print(f"SMILES: {smiles}")
print(f"Molecule object: {mol}")
print(f"Number of atoms: {mol.GetNumAtoms()}")
print(f"Number of bonds: {mol.GetNumBonds()}")

In [ ]:
# More complex molecule: Aspirin
aspirin_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
aspirin = Chem.MolFromSmiles(aspirin_smiles)

print(f"Aspirin SMILES: {aspirin_smiles}")
print(f"Number of atoms: {aspirin.GetNumAtoms()}")
print(f"Number of bonds: {aspirin.GetNumBonds()}")

## 4.2 Calculating Molecular Descriptors

In [ ]:
# Calculate Lipinski descriptors for Aspirin
mol = aspirin

mw = Descriptors.MolWt(mol)
logp = Descriptors.MolLogP(mol)
hbd = Lipinski.NumHDonors(mol)
hba = Lipinski.NumHAcceptors(mol)

print("Aspirin Lipinski Descriptors:")
print(f"  Molecular Weight: {mw:.2f} Da (Rule: < 500)")
print(f"  LogP: {logp:.2f} (Rule: < 5)")
print(f"  H-Bond Donors: {hbd} (Rule: ≤ 5)")
print(f"  H-Bond Acceptors: {hba} (Rule: ≤ 10)")

# Check if drug-like
violations = 0
if mw > 500: violations += 1
if logp > 5: violations += 1
if hbd > 5: violations += 1
if hba > 10: violations += 1
print(f"\nLipinski violations: {violations} (Drug-like if ≤ 1)")

## 4.3 Calculate Descriptors for Multiple Molecules

In [ ]:
# Function to calculate Lipinski descriptors
def calculate_lipinski(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None, None, None
    return (
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Lipinski.NumHDonors(mol),
        Lipinski.NumHAcceptors(mol)
    )

# Apply to our dataframe
lipinski_data = df['smiles'].apply(calculate_lipinski)
df['MW'] = [x[0] for x in lipinski_data]
df['LogP'] = [x[1] for x in lipinski_data]
df['HBD'] = [x[2] for x in lipinski_data]
df['HBA'] = [x[3] for x in lipinski_data]

df[['molecule_id', 'smiles', 'MW', 'LogP', 'HBD', 'HBA']]

## 4.4 Visualize Molecules

In [ ]:
# Draw a single molecule
mol = Chem.MolFromSmiles("c1ccccc1")  # Benzene
Draw.MolToImage(mol, size=(200, 200))

In [ ]:
# Draw multiple molecules in a grid
mols = [Chem.MolFromSmiles(s) for s in df['smiles']]
legends = df['molecule_id'].tolist()
Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(200, 200), legends=legends)

---
# 5. Scikit-learn — Machine Learning

Building predictive models.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

print("Scikit-learn imported successfully!")

## 5.1 Prepare Data for ML

In [ ]:
# Features (X) and Target (Y)
# We'll predict pIC50 from molecular descriptors
X = df[['MW', 'LogP', 'HBD', 'HBA']].values
Y = df['pIC50'].values

print("Features (X) shape:", X.shape)
print("Target (Y) shape:", Y.shape)
print("\nFeature matrix:")
print(X)

## 5.2 Train/Test Split

In [ ]:
# Split data: 80% train, 20% test
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

## 5.3 Train a Model

In [ ]:
# Create and train a Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

print("Model trained!")

## 5.4 Make Predictions and Evaluate

In [ ]:
# Predict on test set
Y_pred = model.predict(X_test)

# Calculate metrics
r2 = r2_score(Y_test, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"\nActual vs Predicted:")
for actual, pred in zip(Y_test, Y_pred):
    print(f"  Actual: {actual:.2f}, Predicted: {pred:.2f}")

## 5.5 Cross-Validation

In [ ]:
# 3-fold cross-validation (small dataset, so 3 folds)
cv_scores = cross_val_score(model, X, Y, cv=3, scoring='r2')

print("Cross-Validation Results:")
print(f"  R² scores: {cv_scores}")
print(f"  Mean R²: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

## 5.6 Feature Importance

In [ ]:
# Which features are most important?
feature_names = ['MW', 'LogP', 'HBD', 'HBA']
importances = model.feature_importances_

plt.figure(figsize=(8, 4))
plt.barh(feature_names, importances, color='steelblue')
plt.xlabel('Importance')
plt.title('Feature Importance for pIC50 Prediction')
plt.show()

print("\nFeature Importances:")
for name, imp in zip(feature_names, importances):
    print(f"  {name}: {imp:.4f}")

---
# 6. ChEMBL Web Resource Client — Database Access

ChEMBL is the world's largest open database of bioactive molecules. The `chembl_webresource_client` lets us query it programmatically.

In [ ]:
!uv pip install --system chembl_webresource_client --quiet
from chembl_webresource_client.new_client import new_client
print("ChEMBL client imported!")

## 6.1 Search for Target Proteins

In [ ]:
# Search for a target by name
target = new_client.target
target_query = target.search('beta amyloid')

# Convert to DataFrame
targets_df = pd.DataFrame.from_dict(target_query)
print(f"Found {len(targets_df)} targets related to 'beta amyloid'")
targets_df[['target_chembl_id', 'pref_name', 'target_type', 'organism']].head()

## 6.2 Get Bioactivity Data for a Target

In [ ]:
# Get bioactivity data for a specific target
# CHEMBL2487 = Beta amyloid A4 protein
activity = new_client.activity
bioactivities = activity.filter(target_chembl_id='CHEMBL2487').filter(standard_type='IC50')

# This can be slow, so let's just get first 10 for demo
bioactivities_list = list(bioactivities[:10])
print(f"Retrieved {len(bioactivities_list)} bioactivity records")

# Convert to DataFrame
bio_df = pd.DataFrame.from_dict(bioactivities_list)
bio_df[['molecule_chembl_id', 'canonical_smiles', 'standard_value', 'standard_units']].head()

## 6.3 Search for Molecules

In [ ]:
# Search for a specific molecule by name
molecule = new_client.molecule
aspirin = molecule.search('aspirin')[0]

print("Aspirin from ChEMBL:")
print(f"  ChEMBL ID: {aspirin['molecule_chembl_id']}")
print(f"  Name: {aspirin['pref_name']}")
print(f"  SMILES: {aspirin['molecule_structures']['canonical_smiles']}")
print(f"  MW: {aspirin['molecule_properties']['full_mwt']}")

---
# 7. OpenPyXL — Excel File Handling

Pandas uses openpyxl to read/write Excel files (.xlsx).

In [ ]:
!uv pip install --system openpyxl --quiet
import openpyxl
print(f"openpyxl version: {openpyxl.__version__}")

## 7.1 Read Excel Files with Pandas

In [ ]:
# Create a sample Excel file first
sample_data = pd.DataFrame({
    'Molecule': ['Aspirin', 'Ibuprofen', 'Caffeine'],
    'MW': [180.16, 206.29, 194.19],
    'IC50_nM': [500, 1200, 8000]
})
sample_data.to_excel('sample_drugs.xlsx', index=False, sheet_name='Drugs')
print("Created sample_drugs.xlsx")

# Read it back
df_excel = pd.read_excel('sample_drugs.xlsx', sheet_name='Drugs')
df_excel

## 7.2 Direct openpyxl Usage (Advanced)

In [ ]:
# Load workbook directly
wb = openpyxl.load_workbook('sample_drugs.xlsx')
print(f"Sheet names: {wb.sheetnames}")

# Access a specific sheet
sheet = wb['Drugs']
print(f"\nCell A1: {sheet['A1'].value}")
print(f"Cell B2: {sheet['B2'].value}")

# Iterate through rows
print("\nAll data:")
for row in sheet.iter_rows(values_only=True):
    print(row)

---
# 8. SciPy — Statistical Tests

SciPy provides statistical functions like the Mann-Whitney U test.

In [ ]:
from scipy import stats
print("SciPy stats imported!")

## 8.1 Mann-Whitney U Test

A non-parametric test to compare two independent groups. Used to check if active and inactive compounds have significantly different properties.

In [ ]:
# Create sample data: pIC50 values for active vs inactive compounds
active_pIC50 = np.array([7.5, 8.0, 7.2, 8.5, 7.8, 8.2, 7.0, 8.8])
inactive_pIC50 = np.array([4.5, 4.8, 5.0, 4.2, 4.9, 5.2, 4.0, 4.7])

print("Active pIC50:", active_pIC50)
print("Inactive pIC50:", inactive_pIC50)
print(f"\nActive mean: {active_pIC50.mean():.2f}")
print(f"Inactive mean: {inactive_pIC50.mean():.2f}")

In [ ]:
# Perform Mann-Whitney U test
statistic, p_value = stats.mannwhitneyu(active_pIC50, inactive_pIC50)

print("Mann-Whitney U Test Results:")
print(f"  U-statistic: {statistic}")
print(f"  P-value: {p_value:.2e}")
print(f"\nInterpretation:")
if p_value < 0.05:
    print("  ✓ Significant difference (p < 0.05)")
    print("  Active and inactive compounds have different pIC50 distributions")
else:
    print("  ✗ No significant difference (p >= 0.05)")

## 8.2 Other Useful Statistical Functions

In [ ]:
# Pearson correlation
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 4, 5])
corr, p = stats.pearsonr(x, y)
print(f"Pearson correlation: r={corr:.3f}, p={p:.3f}")

# T-test (parametric alternative to Mann-Whitney)
t_stat, t_p = stats.ttest_ind(active_pIC50, inactive_pIC50)
print(f"\nT-test: t={t_stat:.3f}, p={t_p:.2e}")

# Shapiro-Wilk test for normality
stat, p = stats.shapiro(active_pIC50)
print(f"\nShapiro-Wilk (active): W={stat:.3f}, p={p:.3f}")
print(f"  Normal distribution: {'Yes' if p > 0.05 else 'No'}")

---
# 9. PaDELPy — Molecular Fingerprints

PaDEL calculates molecular fingerprints (881-bit PubChem fingerprints) from SMILES.

In [ ]:
!uv pip install --system padelpy --quiet
from padelpy import from_smiles
print("PaDELPy imported!")
print("Note: PaDEL requires Java to be installed")

## 9.1 Calculate Fingerprints from SMILES

In [ ]:
# Check if Java is available
import subprocess
try:
    result = subprocess.run(['java', '-version'], capture_output=True, text=True)
    print("Java is available!")
    java_available = True
except:
    print("Java not found - PaDEL won't work without it")
    java_available = False

In [ ]:
# Calculate fingerprints for a molecule (if Java available)
if java_available:
    try:
        # This calculates PubChem fingerprints
        smiles = "CCO"  # Ethanol
        descriptors = from_smiles(smiles, fingerprints=True, descriptors=False)
        
        print(f"SMILES: {smiles}")
        print(f"Number of fingerprint bits: {len(descriptors)}")
        print(f"First 10 fingerprint values: {list(descriptors.values())[:10]}")
    except Exception as e:
        print(f"PaDEL error: {e}")
        print("This is normal if Java/PaDEL isn't properly configured")
else:
    print("Skipping PaDEL demo - Java not available")
    print("\nIn the main notebook, we use pre-computed fingerprints from a CSV file")

## 9.2 Understanding Fingerprints

Molecular fingerprints are binary vectors (0s and 1s) representing molecular substructures:

| Bit | Meaning | Example |
|-----|---------|---------|
| 0 | Has ≥1 Carbon | Most organic molecules |
| 1 | Has ≥2 Carbons | Ethanol (CCO) |
| ... | ... | ... |
| 115 | Has benzene ring | Aspirin |
| ... | ... | ... |
| 880 | Specific SMARTS pattern | Complex substructure |

**Why fingerprints?**
- Convert molecular structure to numbers for ML
- Each bit represents presence/absence of a substructure
- 881 bits = 881 features for machine learning

---
# 10. LazyPredict — Automated Model Benchmarking

LazyPredict runs 30+ ML algorithms automatically and ranks them by performance.

In [ ]:
!uv pip install --system lazypredict xgboost lightgbm --quiet
from lazypredict.Supervised import LazyRegressor
print("LazyPredict imported!")

## 10.1 Quick Model Comparison

In [ ]:
# Create sample dataset
np.random.seed(42)
n_samples = 100

# Features: MW, LogP, HBD, HBA (simulated)
X_demo = np.random.rand(n_samples, 4) * np.array([300, 5, 5, 10]) + np.array([200, 0, 0, 0])
# Target: pIC50 (simulated with some relationship to features)
Y_demo = 5 + 0.005 * X_demo[:, 0] - 0.3 * X_demo[:, 1] + np.random.randn(n_samples) * 0.5

print(f"Features shape: {X_demo.shape}")
print(f"Target shape: {Y_demo.shape}")

In [ ]:
# Split data
X_train_demo, X_test_demo, Y_train_demo, Y_test_demo = train_test_split(
    X_demo, Y_demo, test_size=0.2, random_state=42
)

# Run LazyPredict
import warnings
warnings.filterwarnings('ignore')

reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = reg.fit(X_train_demo, X_test_demo, Y_train_demo, Y_test_demo)

print("Top 10 Models by R² Score:")
models.head(10)

## 10.2 Understanding LazyPredict Output

| Column | Meaning |
|--------|---------|
| Adjusted R-Squared | R² adjusted for number of features |
| R-Squared | Proportion of variance explained (0-1) |
| RMSE | Root Mean Squared Error (lower = better) |
| Time Taken | Training time in seconds |

**Key insight:** LazyPredict helps you quickly identify which algorithms work best for your data before spending time on hyperparameter tuning.

---
# 11. Cleanup

In [ ]:
# Remove temporary files
import os
if os.path.exists('sample_bioactivity.csv'):
    os.remove('sample_bioactivity.csv')
    print("Cleaned up temporary files!")

---
# 12. Quick Reference

## NumPy
```python
import numpy as np
arr = np.array([1, 2, 3])
np.mean(arr), np.std(arr), np.log10(arr)
```

## Pandas
```python
import pandas as pd
df = pd.read_csv('file.csv')
df.head(), df.describe(), df['column']
df[df['column'] > value]  # filtering
df.groupby('column').mean()
```

## Matplotlib/Seaborn
```python
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x='class', y='value')
plt.show()
```

## RDKit
```python
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
mol = Chem.MolFromSmiles('CCO')
mw = Descriptors.MolWt(mol)
logp = Descriptors.MolLogP(mol)
```

## Scikit-learn
```python
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
model = RandomForestRegressor()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
```

---

**Next Step:** You're now ready for the main **Alzheimer's Drug Discovery** notebook!